## install moveit_commander

```
sudo apt-get install ros-"DISTRO"-moveit-commander
pip install pyassimp
```

## reference

https://ros-planning.github.io/moveit_tutorials/doc/move_group_python_interface/move_group_python_interface_tutorial.html

## Before Running Moveit 

```
# run gazebo
roslaunch ur_gazebo ur5_robotiq.launch sim:=true limited:=true
# run moveit
roslaunch ur5_moveit_config ur5_moveit_planning_execution.launch sim:=true limited:=true
# run rviz
roslaunch ur5_moveit_config moveit_rviz.launch config:=true
```


In [1]:
import os
ROS_DISTRO = os.environ['ROS_DISTRO']

In [2]:
from __future__ import print_function
from six.moves import input

import sys
ros_python_path = '/opt/ros/{}/lib/python2.7/dist-packages'.format(ROS_DISTRO)
if not ros_python_path in sys.path:
    sys.path.append(ros_python_path)

import rospy
import copy

In [3]:
#for moveit
from moveit_commander.conversions import pose_to_list
from std_msgs.msg import String
import geometry_msgs.msg
import moveit_commander
import moveit_msgs.msg
from math import pi

# for gripper
from robotiq_2f_gripper_msgs.msg import CommandRobotiqGripperFeedback, CommandRobotiqGripperResult, CommandRobotiqGripperAction, CommandRobotiqGripperGoal
from robotiq_2f_gripper_control.robotiq_2f_gripper_driver import Robotiq2FingerGripperDriver as Robotiq
import actionlib

## initialize moveit_commander

In [4]:
moveit_commander.roscpp_initialize(sys.argv)
rospy.init_node('arm_ur5_gripper_robotiq_client', anonymous=True)

In [5]:
# Provides information such as the robot’s kinematic model and the robot’s current joint states
robot = moveit_commander.RobotCommander()

In [6]:
'''
print(robot.get_group_names())
print(robot.get_current_state())
print(robot.get_joint_names())
'''

'\nprint(robot.get_group_names())\nprint(robot.get_current_state())\nprint(robot.get_joint_names())\n'

In [7]:
# provides a remote interface for getting, setting, and updating the robot’s internal understanding of the surrounding world
# 플래닝할때, 주변 장애물을 반영할 수 있게, 아래의 object를 통해 환경에 장애물을 추가할 수 있다.
scene = moveit_commander.PlanningSceneInterface()

In [8]:
# plan 한 trajectory를 rviz화면에 시뮬레이션해줌.
# "roslaunch ur5_moveit_config moveit_rviz.launch config:=true" 을 통해 미리 rviz를 띄어놔야 함.
display_trajectory_publisher = rospy.Publisher('/move_group/display_planned_path',
                                               moveit_msgs.msg.DisplayTrajectory,
                                               queue_size=20)

In [9]:
# move_group 객체를 통해 각 joint(move_element)의 desired states를 지정할 수 있고, moveit은 이 값을 바탕으로 planning을 해줌.
group_name = "manipulator"
move_group = moveit_commander.MoveGroupCommander(group_name)

In [10]:
# We can get the name of the reference frame for this robot:
planning_frame = move_group.get_planning_frame()
print("============ Planning frame: %s" % planning_frame)

# We can also print the name of the end-effector link for this group:
eef_link = move_group.get_end_effector_link()
print("============ End effector link: %s" % eef_link)

# We can get a list of all the groups in the robot:
group_names = robot.get_group_names()
print("============ Available Planning Groups:", robot.get_group_names())

# Sometimes for debugging it is useful to print the entire state of the
# robot:
print("============ Printing robot state")
print(robot.get_current_state())
print("")

============ Planning frame: world
============ End effector link: ee_link
============ Available Planning Groups: ['endeffector', 'manipulator']
============ Printing robot state
joint_state: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "world"
  name: 
    - shoulder_pan_joint
    - shoulder_lift_joint
    - elbow_joint
    - wrist_1_joint
    - wrist_2_joint
    - wrist_3_joint
    - finger_joint
    - left_inner_finger_joint
    - left_inner_knuckle_joint
    - right_inner_knuckle_joint
    - right_outer_knuckle_joint
    - right_inner_finger_joint
  position: [-0.017814759014429526, 0.37038988736662315, 2.468016389776178, -0.6287761067335342, 1.9685016752487456, 2.6919361917664286, 0.004274148940518252, -0.004274148940518252, 0.004274148940518252, 0.004274148940518252, 0.004274148940518252, -0.004274148940518252]
  velocity: []
  effort: []
multi_dof_joint_state: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    fr

In [11]:
move_group.get_current_joint_values()

[-0.01781289397662622,
 0.37025260736700893,
 2.4680922516031742,
 -0.6283789977573662,
 1.9804679298693895,
 2.69191043893062]

In [12]:
move_group.set_max_velocity_scaling_factor = 0.001

In [13]:
move_group.set_max_velocity_scaling_factor = 0.1
joint_goal = move_group.get_current_joint_values()
joint_goal[0] = 0
joint_goal[1] = 0
joint_goal[2] = 0
joint_goal[3] = 0
joint_goal[4] = 0
joint_goal[5] = 0

# The go command can be called with joint values, poses, or without any
# parameters if you have already set the pose or joint target for the group
move_group.go(joint_goal, wait=True)

# Calling ``stop()`` ensures that there is no residual movement
move_group.stop()

In [14]:
move_group.set_max_velocity_scaling_factor = 0.01
joint_goal = move_group.get_current_joint_values()

joint_goal = [2.928491837446625,
 -1.3835698676261812,
 1.716120040745281,
 -2.0037368618694433,
 -1.5245347859526337,
 2.887431079190039]

# The go command can be called with joint values, poses, or without any
# parameters if you have already set the pose or joint target for the group
move_group.go(joint_goal, wait=True)

# Calling ``stop()`` ensures that there is no residual movement
move_group.stop()

In [17]:
move_group.set_max_velocity_scaling_factor = 0.01
joint_goal = move_group.get_current_joint_values()
'''
joint_goal = [2.9412768042098367,
 -1.1789994888028392,
 1.8574884240683716,
 -2.35008618243274,
 -1.5256609547041684,
 2.900080107538237]
'''
joint_goal2 = [2.9582129039798524,
 -1.1426872994651491,
 1.781375071390455,
 -2.2244975444512622,
 -1.516971398631207,
 2.9191883264198317]
joint_goal[1] = joint_goal2[1]

move_group.go(joint_goal, wait=True)
move_group.stop()

In [15]:
action_name = rospy.get_param('~action_name', 'command_robotiq_action')
print(action_name)

command_robotiq_action


In [16]:
robotiq_client = actionlib.SimpleActionClient(action_name, CommandRobotiqGripperAction)
robotiq_client.wait_for_server()
print("Client test: Starting sending goals")

Client test: Starting sending goals


In [19]:
## Manually set all the parameters of the gripper goal state.
######################################################################################

goal = CommandRobotiqGripperGoal()
goal.emergency_release = False
goal.stop = False
goal.position = 0.8
goal.speed = 0.1
goal.force = 1.0

# Sends the goal to the gripper.
robotiq_client.send_goal(goal)
# Block processing thread until gripper movement is finished, comment if waiting is not necesary.
robotiq_client.wait_for_result()

True

In [18]:
## Manually set all the parameters of the gripper goal state.
######################################################################################

goal = CommandRobotiqGripperGoal()
goal.emergency_release = False
goal.stop = False
goal.position = 0.0
goal.speed = 0.1
goal.force = 5.0

# Sends the goal to the gripper.
robotiq_client.send_goal(goal)
# Block processing thread until gripper movement is finished, comment if waiting is not necesary.
robotiq_client.wait_for_result()

True

In [15]:
joint_goal = move_group.get_current_joint_values()
joint_goal[0] = 0
joint_goal[1] = -pi/2
joint_goal[2] = -pi/2
joint_goal[3] = 0
joint_goal[4] = 0
joint_goal[5] = 0

move_group.go(joint_goal, wait=True)

move_group.stop()

goal = CommandRobotiqGripperGoal()
goal.position = 0.0
goal.speed = 1.0

robotiq_client.send_goal(goal)
robotiq_client.wait_for_result()

True

In [23]:
joint_goal = move_group.get_current_joint_values()
joint_goal[0] = pi/2
joint_goal[1] = -pi/2
joint_goal[2] = -pi/2
joint_goal[3] = 0
joint_goal[4] = pi/2
joint_goal[5] = 0

move_group.go(joint_goal, wait=True)

move_group.stop()

goal = CommandRobotiqGripperGoal()
goal.position = 0.6
goal.speed = 1.0

robotiq_client.send_goal(goal)
robotiq_client.wait_for_result()

True